In [1]:
from abia_energia import*
from __future__ import annotations
from typing import List, Generator, Set
import timeit
from search import Problem, hill_climbing
from math import*
from collections import*


In [2]:
centrales = Centrales([4, 4, 4], 42)

In [3]:
centrales

[Central(tipo=0|produccion=569|cx=3|cy=94),
 Central(tipo=0|produccion=387|cx=28|cy=17),
 Central(tipo=0|produccion=618|cx=86|cy=94),
 Central(tipo=0|produccion=696|cx=11|cy=75),
 Central(tipo=1|produccion=163|cx=3|cy=11),
 Central(tipo=1|produccion=132|cx=64|cy=77),
 Central(tipo=1|produccion=103|cx=25|cy=91),
 Central(tipo=1|produccion=197|cx=69|cy=53),
 Central(tipo=2|produccion=29|cx=75|cy=35),
 Central(tipo=2|produccion=82|cx=0|cy=97),
 Central(tipo=2|produccion=82|cx=89|cy=54),
 Central(tipo=2|produccion=40|cx=19|cy=27)]

In [4]:
clientes = Clientes(300, [0.2, 0.3, 0.5], 0.5, 42)

In [5]:
class ProblemParameters(object):
    def __init__(self, num_centrals: int, num_clients: int, clients: list, centrals: list):
        self.num_centrals = num_centrals # Màxim número de contenidors
        self.num_clients = num_clients # Màxim número de paquets
        self.centrals = centrals # Llista d'objectes central
        self.clients = clients # Llista d'objectes clients
        
        self.ingresos=0
        for client in self.clients:
            self.ingresos += (VEnergia.tarifa_cliente_no_garantizada(client.Tipo) if client.Contrato else VEnergia.tarifa_cliente_garantizada(client.Tipo))*client.Consumo
            
        self.matriu_c=[[None for j in range(self.num_centrals)] for i in range(self.num_clients)]
                
        for i in range(self.num_clients): 
            for j in range(self.num_centrals):
                x,y=clientes[i].CoordX, clientes[i].CoordY
                xc,yc=centrales[j].CoordX,centrales[j].CoordY
                distancia=dist([x,y],[xc,yc])

                consumo=clientes[i].Consumo
                consumoT= consumo + consumo*VEnergia.loss(distancia)
                self.matriu_c[i][j]= consumoT

        
    def __repr__(self):
        return f"Params(centrals={self.centrals}, clients={self.clients})"
        
class EnergyOperator(object):
    pass

class MoveClient(EnergyOperator):
    def __init__(self, client_i: int, central_j: int, central_k: int):
        self.client_i = client_i
        self.central_j = central_j
        self.central_k = central_k
    
    def get(self):
        return self.client_i ,self.central_j, self.central_k
        
    def __repr__(self) -> str:
        return f"Move client {self.client_i} from central {self.central_j} to central {self.central_k}"
    
class SwapClient(EnergyOperator):
    def __init__(self, client_i: int, client_j: int):
        self.client_i = client_i
        self.client_j = client_j
        
    def __repr__(self) -> str:
        return f"Swap clients {self.client_i} and {self.client_j}"
    
class StateRepresentation(object):
    def __init__(self, params: ProblemParameters, v_a: List[int], produccion_central: List[int] = None, benefici: float = None):
        self.params = params
        self.v_a = v_a
        self.benefici = benefici
        
        if produccion_central is None:
            self.produccion_central=[0 for j in range(self.params.num_centrals)]

            for client_i, central_j in enumerate(self.v_a):
                consum_client_i = self.params.matriu_c[client_i][central_j]
                self.produccion_central[central_j] += consum_client_i
        else:
            self.produccion_central = produccion_central
        
    def copy(self) -> StateRepresentation:
        return StateRepresentation(self.params, self.v_a.copy(), self.produccion_central.copy(), self.benefici)
    
    def __repr__(self) -> str:
        return f"v_a={self.v_a} | Produccion_central={self.produccion_central} | {self.params}"
    
    def generate_actions(self) -> Generator[EnergyOperator,None,None]:
        
        for client_i, central_j in enumerate(self.v_a):     
            consumo_cliente_i= self.params.matriu_c[client_i][central_j]
            for central_k, prod_i in enumerate(self.produccion_central):
                if central_j != central_k and self.params.centrals[central_j].Produccion - prod_i >= consumo_cliente_i:
                    yield MoveClient(client_i, central_j, central_k)
                    
        # Intercanviar paquets
        for client_i in range(self.params.num_clients):
            for client_j in range(client_i+1, self.params.num_clients):
                if client_i != client_j:
                    central_i = self.v_a[client_i]
                    central_j = self.v_a[client_j]
                    
                    consumt_i_i =  self.params.matriu_c[client_i][central_i]
                    consumt_j_j =  self.params.matriu_c[client_j][central_j]
                    
                    # Condició: hi ha espai lliure suficient per fer l'intercanvi
                    # (Espai lliure del contenidor + espai que deixa el paquet >= espai del nou paquet)
                    if self.produccion_central[central_i] + consumt_j_j >= self.produccion_central[central_i] + consumt_i_i and self.produccion_central[central_j] + consumt_i_i >= self.produccion_central[central_j] + consumt_j_j:
                        pass
                        #yield SwapClient(client_i, client_j)
                        
        #Afegir client
             
    def apply_action(self, action: EnergyOperator) -> StateRepresentation:
        new_state = self.copy()
        if isinstance(action, MoveClient):
            # action
            client_i =  action.client_i
            central_j = action.central_j
            central_k = action.central_k
            
            new_state.v_a[client_i] = central_k
            
            new_state.produccion_central[central_j] -= self.params.matriu_c[client_i][central_j]
            new_state.produccion_central[central_k] += self.params.matriu_c[client_i][central_k]
            
            # agafar benefici pare
            
            benefici_pare=self.heuristic()
            
            
            # calcular com varia
            
            obj_client_canviat= self.params.clients[client_i]
            obj_central_anterior = self.params.centrals[central_j]
            obj_central_nova = self.params.centrals[central_k]          

            
            benefici_fill = benefici_pare 
            
            
            # cost diari i cost de parada
                        
            if len(set(new_state.v_a))> len(set(self.v_a)): 
                
                central_tipo=new_state.params.centrals[central_j].Tipo
                coste_produccion_megaw = new_state.params.centrals[central_j].Produccion * VEnergia.costs_production_mw(central_tipo)
                benefici_fill += (coste_produccion_megaw)+(VEnergia.daily_cost(central_tipo))
                benefici_fill -= (VEnergia.stop_cost(central_tipo))
        
                            
                central_tipo=new_state.params.centrals[central_k].Tipo
                coste_produccion_megaw = new_state.params.centrals[central_k].Produccion * VEnergia.costs_production_mw(central_tipo)
                benefici_fill -= (coste_produccion_megaw)+(VEnergia.daily_cost(central_tipo))
                benefici_fill += (VEnergia.stop_cost(central_tipo))
                
            elif len(set(new_state.v_a))< len(set(self.v_a)): 
                
                central_tipo=new_state.params.centrals[central_j].Tipo
                coste_produccion_megaw = new_state.params.centrals[central_j].Produccion * VEnergia.costs_production_mw(central_tipo)
                benefici_fill -= (coste_produccion_megaw)+(VEnergia.daily_cost(central_tipo))
                benefici_fill += (VEnergia.stop_cost(central_tipo))
                
                central_tipo=new_state.params.centrals[central_k].Tipo
                coste_produccion_megaw = new_state.params.centrals[central_k].Produccion * VEnergia.costs_production_mw(central_tipo)
                benefici_fill += (coste_produccion_megaw)+(VEnergia.daily_cost(central_tipo))
                benefici_fill -= (VEnergia.stop_cost(central_tipo))
            
            
            
            
            # assignar heuristic fill
            print(benefici_pare, benefici_fill, set(new_state.v_a).symmetric_difference(set(self.v_a)) )
            
            new_state.benefici = benefici_fill
            
        
           
                  
                
        elif isinstance(action, SwapClient):
            
        
            client_i = action.client_i
            client_j = action.client_j
            
            central_k = new_state.v_a[client_i]
            central_l = new_state.v_a[client_j]
            
            new_state.v_a[client_i] = central_k
            new_state.v_a[client_j] = central_l
            
            new_state.produccion_central[central_k] -= self.params.matriu_c[client_i][central_k]
            new_state.produccion_central[central_l] -= self.params.matriu_c[client_j][central_l]
            new_state.produccion_central[central_k] += self.params.matriu_c[client_j][central_k]
            new_state.produccion_central[central_l] += self.params.matriu_c[client_i][central_l]
                            
            benefici_fill=1000
            
             
            
        return new_state
    
    def heuristic(self) -> float:
        return self.benefici
        
        
        
    
class EnergyProblem(Problem):
    def __init__(self, initial_state: StateRepresentation):
        super().__init__(initial_state)
        
    def actions(self, state: StateRepresentation) -> Generator[EnergyOperator,None,None]:
        return state.generate_actions()
    
    def result(self, state: StateRepresentation, action: EnergyOperator) -> StateRepresentation:
        return state.apply_action(action)
    
    def value(self, state: StateRepresentation) -> float:
        
        return  state.heuristic()
        
    def goal_test(self, state: StateRepresentation) -> bool:
        return False

In [6]:
def generate_initial_state(params: ProblemParameters) -> StateRepresentation:    
    v_a=[]
    index_central = 0
    for client_i in params.clients:
        v_a.append(index_central)
    return StateRepresentation(params, v_a,  benefici=0)

In [7]:
def generate_initial_state2(params: ProblemParameters) -> StateRepresentation:    
    v_a=[]    
    for client_i in range(params.num_clients):    
        index_central= params.matriu_c[client_i].index(min(params.matriu_c[client_i]))
        v_a.append(index_central)
        
    return StateRepresentation(params, v_a,benefici=0)

In [8]:
def generate_initial_state3(params: ProblemParameters) -> StateRepresentation:    
    v_a=[]    
    for client_i in range(params.num_clients):    
        index_central= params.matriu_c[client_i].index(max(params.matriu_c[client_i]))
        v_a.append(index_central)
        
    return StateRepresentation(params, v_a, benefici=0)

In [9]:
def generate_initial_state6(params: ProblemParameters) -> StateRepresentation:
    # afegir index per saber de quina central estem parlant
    centrals_tuples = [(i, central) for i, central in zip(range(params.num_centrals), params.centrals)]
    
    # ordenar tuples (index, objecteCentral) en funció de objecteCentral.Produccion
    # reverse = True implica ordenar les centrals de més producció a menys
    centrals_ordenades = sorted(centrals_tuples, key=lambda x: x[1].Produccion, reverse=True)
    
    plena_fins = 1/2
    # fem associacions de clients per ordre de centrals_ordenades fins que cada central estigui plena fins (variable plena_fins)
    index_ordenades, prod_max = 0, centrals_ordenades[0][1].Produccion
    comptador_produccio = 0
    v_a = []
    
    for client in params.clients:
        # si afegint el client passem de la meitat
        if comptador_produccio + client.Consumo > prod_max * plena_fins:
            # si queden més centrals
            if index_ordenades < params.num_centrals - 1:
                # movem l'index de centrals_ordenades a la següent central, que segur que estarà buida
                index_ordenades += 1
                prod_max = centrals_ordenades[index_ordenades][1].Produccion
                comptador_produccio = 0
            else:
                raise ValueError(f"Omplint totes les centrals fins al {plena_fins*100}% t'acaben faltant centrals")
        
        v_a.append(centrals_ordenades[index_ordenades][0])
        comptador_produccio += client.Consumo
    
    return StateRepresentation(params, v_a, benefici=0)

In [10]:
"""def generate_initial_state4(params: ProblemParameters) -> StateRepresentation:    
    v_a=[] 
    consum=0
    central_ocupada=[]
    for client_i in range(params.num_clients):
        centrals_sort_lista= sorted(params.matriu_c[client_i])
        central_dedo=0
        central_index_matriu=params.matriu_c[client_i].index(centrals_sort_lista[central_dedo])

        
        while not central_index_matriu in central_ocupada or central_index_matriu.Produccion/2 < consum+ (params.matriu_c[client_i][central_index_matriu]):
            central_dedo+=1
            central_index_matriu=params.matriu_c[client_i].index(centrals_sort_lista[central_dedo])
            
            
        else:
            while central_index.index(centrals[central_dedo]) in central_ocupada and central_actual<len(params.matriu_c[client_i]):
                central_acutal+=1
                
            index_central= params.matriu_c[client_i].index(centrals[central_actual]) 
        
        v_a.append(index_central)

        
    return StateRepresentation(params, v_a)"""

'def generate_initial_state4(params: ProblemParameters) -> StateRepresentation:    \n    v_a=[] \n    consum=0\n    central_ocupada=[]\n    for client_i in range(params.num_clients):\n        centrals_sort_lista= sorted(params.matriu_c[client_i])\n        central_dedo=0\n        central_index_matriu=params.matriu_c[client_i].index(centrals_sort_lista[central_dedo])\n\n        \n        while not central_index_matriu in central_ocupada or central_index_matriu.Produccion/2 < consum+ (params.matriu_c[client_i][central_index_matriu]):\n            central_dedo+=1\n            central_index_matriu=params.matriu_c[client_i].index(centrals_sort_lista[central_dedo])\n            \n            \n        else:\n            while central_index.index(centrals[central_dedo]) in central_ocupada and central_actual<len(params.matriu_c[client_i]):\n                central_acutal+=1\n                \n            index_central= params.matriu_c[client_i].index(centrals[central_actual]) \n        \n      

In [11]:
initial_state = generate_initial_state6(ProblemParameters(len(centrales), len(clientes), clientes, centrales))

In [12]:
execucions = 1
n_count = hill_climbing(EnergyProblem(initial_state))
t_count = timeit.timeit(lambda: hill_climbing(EnergyProblem(initial_state)), number=execucions)

print(f"heuristic_count,   benefici : {n_count.heuristic()}")
print(f"heuristic_count,   temps/{execucions} execs.  : {t_count}s.")

0 0 set()
0 0 set()
0 0 set()
0 2176 {4}
0 2424 {5}
0 2656 {6}
0 0 set()
0 3195 {8}
0 2400 {9}
0 2400 {10}
0 3030 {11}
0 0 set()
0 0 set()
0 0 set()
0 2176 {4}
0 2424 {5}
0 2656 {6}
0 0 set()
0 3195 {8}
0 2400 {9}
0 2400 {10}
0 3030 {11}
0 0 set()
0 0 set()
0 0 set()
0 2176 {4}
0 2424 {5}
0 2656 {6}
0 0 set()
0 3195 {8}
0 2400 {9}
0 2400 {10}
0 3030 {11}
0 0 set()
0 0 set()
0 0 set()
0 2176 {4}
0 2424 {5}
0 2656 {6}
0 0 set()
0 3195 {8}
0 2400 {9}
0 2400 {10}
0 3030 {11}
0 0 set()
0 0 set()
0 0 set()
0 2176 {4}
0 2424 {5}
0 2656 {6}
0 0 set()
0 3195 {8}
0 2400 {9}
0 2400 {10}
0 3030 {11}
0 0 set()
0 0 set()
0 0 set()
0 2176 {4}
0 2424 {5}
0 2656 {6}
0 0 set()
0 3195 {8}
0 2400 {9}
0 2400 {10}
0 3030 {11}
0 0 set()
0 0 set()
0 0 set()
0 2176 {4}
0 2424 {5}
0 2656 {6}
0 0 set()
0 3195 {8}
0 2400 {9}
0 2400 {10}
0 3030 {11}
0 0 set()
0 0 set()
0 0 set()
0 2176 {4}
0 2424 {5}
0 2656 {6}
0 0 set()
0 3195 {8}
0 2400 {9}
0 2400 {10}
0 3030 {11}
0 0 set()
0 0 set()
0 0 set()
0 2176 {4}
0 2424 

0 0 set()
0 1786 {4}
0 2034 {5}
0 2266 {6}
0 0 set()
0 2805 {8}
0 2010 {9}
0 2010 {10}
0 2640 {11}
0 0 set()
0 0 set()
0 0 set()
0 1786 {4}
0 2034 {5}
0 2266 {6}
0 0 set()
0 2805 {8}
0 2010 {9}
0 2010 {10}
0 2640 {11}
0 0 set()
0 0 set()
0 0 set()
0 1786 {4}
0 2034 {5}
0 2266 {6}
0 0 set()
0 2805 {8}
0 2010 {9}
0 2010 {10}
0 2640 {11}
0 0 set()
0 0 set()
0 0 set()
0 1786 {4}
0 2034 {5}
0 2266 {6}
0 0 set()
0 2805 {8}
0 2010 {9}
0 2010 {10}
0 2640 {11}
0 0 set()
0 0 set()
0 0 set()
0 1786 {4}
0 2034 {5}
0 2266 {6}
0 0 set()
0 2805 {8}
0 2010 {9}
0 2010 {10}
0 2640 {11}
0 0 set()
0 0 set()
0 0 set()
0 1786 {4}
0 2034 {5}
0 2266 {6}
0 0 set()
0 2805 {8}
0 2010 {9}
0 2010 {10}
0 2640 {11}
0 0 set()
0 0 set()
0 0 set()
0 1786 {4}
0 2034 {5}
0 2266 {6}
0 0 set()
0 2805 {8}
0 2010 {9}
0 2010 {10}
0 2640 {11}
0 0 set()
0 0 set()
0 0 set()
0 1786 {4}
0 2034 {5}
0 2266 {6}
0 0 set()
0 2805 {8}
0 2010 {9}
0 2010 {10}
0 2640 {11}
0 0 set()
0 0 set()
0 0 set()
0 1786 {4}
0 2034 {5}
0 2266 {6}
0 0 s

0 2560 {8}
0 1765 {9}
0 1765 {10}
0 2395 {11}
0 0 set()
0 0 set()
0 0 set()
0 1541 {4}
0 1789 {5}
0 2021 {6}
0 0 set()
0 2560 {8}
0 1765 {9}
0 1765 {10}
0 2395 {11}
0 631 {4}
0 879 {5}
0 1111 {6}
0 0 set()
0 1650 {8}
0 855 {9}
0 855 {10}
0 1485 {11}
0 0 set()
0 631 {4}
0 879 {5}
0 1111 {6}
0 0 set()
0 1650 {8}
0 855 {9}
0 855 {10}
0 1485 {11}
0 0 set()
0 631 {4}
0 879 {5}
0 1111 {6}
0 0 set()
0 1650 {8}
0 855 {9}
0 855 {10}
0 1485 {11}
0 631 {4}
0 879 {5}
0 1111 {6}
0 0 set()
0 1650 {8}
0 855 {9}
0 855 {10}
0 1485 {11}
0 0 set()
0 631 {4}
0 879 {5}
0 1111 {6}
0 0 set()
0 1650 {8}
0 855 {9}
0 855 {10}
0 1485 {11}
0 0 set()
0 631 {4}
0 879 {5}
0 1111 {6}
0 0 set()
0 1650 {8}
0 855 {9}
0 855 {10}
0 1485 {11}
0 0 set()
0 631 {4}
0 879 {5}
0 1111 {6}
0 0 set()
0 1650 {8}
0 855 {9}
0 855 {10}
0 1485 {11}
0 0 set()
0 631 {4}
0 879 {5}
0 1111 {6}
0 0 set()
0 1650 {8}
0 855 {9}
0 855 {10}
0 1485 {11}
0 0 set()
0 631 {4}
0 879 {5}
0 1111 {6}
0 0 set()
0 1650 {8}
0 855 {9}
0 855 {10}
0 1485 {11}


3195 5371 {4}
3195 5619 {5}
3195 5851 {6}
3195 3195 set()
3195 3195 set()
3195 5595 {9}
3195 5595 {10}
3195 6225 {11}
3195 3195 set()
3195 3195 set()
3195 3195 set()
3195 5371 {4}
3195 5619 {5}
3195 5851 {6}
3195 3195 set()
3195 3195 set()
3195 5595 {9}
3195 5595 {10}
3195 6225 {11}
3195 3195 set()
3195 3195 set()
3195 3195 set()
3195 5371 {4}
3195 5619 {5}
3195 5851 {6}
3195 3195 set()
3195 3195 set()
3195 5595 {9}
3195 5595 {10}
3195 6225 {11}
3195 3195 set()
3195 3195 set()
3195 3195 set()
3195 5371 {4}
3195 5619 {5}
3195 5851 {6}
3195 3195 set()
3195 3195 set()
3195 5595 {9}
3195 5595 {10}
3195 6225 {11}
3195 3195 set()
3195 3195 set()
3195 3195 set()
3195 5371 {4}
3195 5619 {5}
3195 5851 {6}
3195 3195 set()
3195 3195 set()
3195 5595 {9}
3195 5595 {10}
3195 6225 {11}
3195 3195 set()
3195 3195 set()
3195 3195 set()
3195 5371 {4}
3195 5619 {5}
3195 5851 {6}
3195 3195 set()
3195 3195 set()
3195 5595 {9}
3195 5595 {10}
3195 6225 {11}
3195 3195 set()
3195 3195 set()
3195 3195 set()
3195

3195 3195 set()
3195 5205 {9}
3195 5205 {10}
3195 5835 {11}
3195 3195 set()
3195 3195 set()
3195 3195 set()
3195 4981 {4}
3195 5229 {5}
3195 5461 {6}
3195 3195 set()
3195 3195 set()
3195 5205 {9}
3195 5205 {10}
3195 5835 {11}
3195 3195 set()
3195 3195 set()
3195 3195 set()
3195 4981 {4}
3195 5229 {5}
3195 5461 {6}
3195 3195 set()
3195 3195 set()
3195 5205 {9}
3195 5205 {10}
3195 5835 {11}
3195 3195 set()
3195 3195 set()
3195 3195 set()
3195 4981 {4}
3195 5229 {5}
3195 5461 {6}
3195 3195 set()
3195 3195 set()
3195 5205 {9}
3195 5205 {10}
3195 5835 {11}
3195 3195 set()
3195 3195 set()
3195 3195 set()
3195 4981 {4}
3195 5229 {5}
3195 5461 {6}
3195 3195 set()
3195 3195 set()
3195 5205 {9}
3195 5205 {10}
3195 5835 {11}
3195 3195 set()
3195 3195 set()
3195 3195 set()
3195 4981 {4}
3195 5229 {5}
3195 5461 {6}
3195 3195 set()
3195 3195 set()
3195 5205 {9}
3195 5205 {10}
3195 5835 {11}
3195 3195 set()
3195 3195 set()
3195 3195 set()
3195 4981 {4}
3195 5229 {5}
3195 5461 {6}
3195 3195 set()
3195

...

22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 2249

22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 2249

22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 22495 set()
22495 2249

23514 23514 set()
23514 23514 set()
23514 23514 set()
23514 23762 {5}
23514 23282 {5}
23514 24034 {5}
23514 23514 {5, 8}
23514 23538 {5}
23514 23538 {5}
23514 22908 {5}
23514 23514 set()
23514 23514 set()
23514 23514 set()
23514 23514 set()
23514 23514 set()
23514 23514 set()
23514 23514 set()
23514 26709 {8}
23514 23514 set()
23514 23514 set()
23514 23514 set()
23514 23514 set()
23514 23514 set()
23514 23514 set()
23514 23514 set()
23514 23514 set()
23514 23514 set()
23514 23514 set()
23514 26709 {8}
23514 23514 set()
23514 23514 set()
23514 23514 set()
23514 23514 set()
23514 23514 set()
23514 23514 set()
23514 23514 set()
23514 23514 set()
23514 23514 set()
23514 23514 set()
23514 26709 {8}
23514 23514 set()
23514 23514 set()
23514 23514 set()
23514 23514 set()
23514 23514 set()
23514 23514 set()
23514 24533 {8}
23514 23514 set()
23514 23514 set()
23514 23514 set()
23514 23994 {6}
23514 23746 {6}
23514 23514 {6, 8}
23514 23770 {6}
23514 23770 {6}
23514 23140 {6}
23514 23514 set()
23

KeyboardInterrupt: 

In [ ]:
print(initial_state.v_a, "\n")
print(n_count.v_a)

In [ ]:
ingresos=0
l=[3, 10, 3, 3, 4, 3, 3, 3, 3, 6, 9, 3, 4, 4, 3, 6, 3, 3, 3, 4, 6, 3, 9, 3, 3, 3, 9, 3, 4, 4, 7, 3, 3, 10, 10, 5, 3, 10, 3, 3, 3, 3, 10, 8, 5, 3, 6, 3, 6, 6, 3, 10, 3, 9, 3, 5, 4, 3, 9, 9, 5, 10, 9, 10, 3, 3, 6, 5, 3, 9, 3, 3, 3, 4, 10, 4, 3, 4, 11, 3, 6, 3, 9, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]
for client in clientes:
    ingresos += (VEnergia.tarifa_cliente_no_garantizada(client.Tipo) if client.Contrato else VEnergia.tarifa_cliente_garantizada(client.Tipo))*client.Consumo

"""for i in set(l):
    central_tipo= centrales[i].Tipo
    coste_produccion_megaw = centrales[i].Produccion * VEnergia.costs_production_mw(central_tipo)
    
    ingresos -= (coste_produccion_megaw)+(VEnergia.daily_cost(central_tipo))"""
ingresos